In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
 
import numpy as np
import tensorflow as tf

# Load from .npz file

In [2]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

# Load NumPy arrays with tf.data.Dataset
Assuming you have an array of examples and a corresponding array of labels, pass the two arrays as a tuple into tf.data.Dataset.from_tensor_slices to create a tf.data.Dataset.

In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples,test_labels))

# Use the datasets
## Shuffle and batch the datasets

In [4]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Build and Train a Model

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['sparse_categorical_accuracy'])

In [7]:
model.fit(train_dataset,epochs=10)

Train for 938 steps
Epoch 1/10
938/938 [==============================] - 3s 3ms/step - loss: 2.8253 - sparse_categorical_accuracy: 0.8580
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.4751 - sparse_categorical_accuracy: 0.9234
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.3677 - sparse_categorical_accuracy: 0.9422
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.3103 - sparse_categorical_accuracy: 0.9506
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2810 - sparse_categorical_accuracy: 0.9562A: 2s - loss:
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2719 - sparse_categorical_accuracy: 0.9616
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2536 - sparse_categorical_accuracy: 0.9638
Epoch 8/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2342 - sparse_categorical_accuracy: 0.9666
Epoch 9/10
938/

In [8]:
model.evaluate(test_dataset)

157/157 [==============================] - 0s 2ms/step - loss: 0.5777 - sparse_categorical_accuracy: 0.9590


[0.5776816975156797, 0.959]

# PANDAS

In [9]:
import pandas as pd

This tutorials uses a small dataset provided by the Cleveland Clinic Foundation for Heart Disease. There are several hundred rows in the CSV. Each row describes a patient, and each column describes an attribute. We will use this information to predict whether a patient has heart disease, which in this dataset is a binary classification task.

In [10]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


In [11]:
df = pd.read_csv(csv_file)

In [12]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [13]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

Convert thal column which is an object in the dataframe to a discrete numerical value.

In [14]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [15]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


# Load data using tf.data.Dataset
Use tf.data.Dataset.from_tensor_slices to read the values from a pandas dataframe.

One of the advantages of using tf.data.Dataset is it allows you to write simple, highly efficient data pipelines. Read the loading data guide to find out more.

In [16]:
target = df.pop('target')

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [18]:
for feat, targ in dataset.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


Since a pd.Series implements the __ array __ protocol it can be used transparently nearly anywhere you would use a np.array or a tf.Tensor.

In [19]:
tf.constant(df['thal'])

<tf.Tensor: shape=(303,), dtype=int32, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3,
     

In [20]:
#the and batch the dataset
train_dataset = dataset.shuffle(len(df)).batch(1)

# Create and train a model

In [21]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [22]:
model = get_compiled_model()
model. fit(train_dataset, epochs=15)

Train for 303 steps
Epoch 1/15
303/303 [==============================] - 1s 3ms/step - loss: 1.4426 - accuracy: 0.6436
Epoch 2/15
303/303 [==============================] - 1s 2ms/step - loss: 0.6505 - accuracy: 0.7327
Epoch 3/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5759 - accuracy: 0.7360
Epoch 4/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5366 - accuracy: 0.7525
Epoch 5/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5219 - accuracy: 0.7492
Epoch 6/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5154 - accuracy: 0.7558
Epoch 7/15
303/303 [==============================] - 1s 2ms/step - loss: 0.4930 - accuracy: 0.7690
Epoch 8/15
303/303 [==============================] - 1s 2ms/step - loss: 0.4643 - accuracy: 0.7690
Epoch 9/15
303/303 [==============================] - 1s 2ms/step - loss: 0.4737 - accuracy: 0.7657
Epoch 10/15
303/303 [==============================] - 1s 2ms/step - loss: 0.452

# Alternative to feature columns
Passing a dictionary as an input to a model is as easy as creating a matching dictionary of tf.keras.layers.Input layers, applying any pre-processing and stacking them up using the functional api. You can use this as an alternative to feature columns.

In [23]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation ='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

The easiest way to preserve the column structure of a pd.DataFrame when used with tf.data is to convert the pd.DataFrame to a dict, and slice that dictionary.

In [24]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [25]:
for dict_slice in dict_slices.take(1):
    print(dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57],
      dtype=int32)>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3], dtype=int32)>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150], dtype=int32)>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168], dtype=int32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int32)>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0], dtype=int32)>, 'thalach': <tf.Tensor: shape=(16

In [26]:
model_func.fit(dict_slices,epochs=15)

Train for 19 steps
Epoch 1/15
19/19 [==============================] - 0s 24ms/step - loss: 1.8146 - accuracy: 0.6964
Epoch 2/15
19/19 [==============================] - 0s 4ms/step - loss: 1.1848 - accuracy: 0.5941
Epoch 3/15
19/19 [==============================] - 0s 4ms/step - loss: 0.9202 - accuracy: 0.6832
Epoch 4/15
19/19 [==============================] - 0s 5ms/step - loss: 0.7700 - accuracy: 0.6733
Epoch 5/15
19/19 [==============================] - 0s 4ms/step - loss: 0.6573 - accuracy: 0.6964
Epoch 6/15
19/19 [==============================] - 0s 4ms/step - loss: 0.5896 - accuracy: 0.7426
Epoch 7/15
19/19 [==============================] - 0s 4ms/step - loss: 0.5638 - accuracy: 0.7459
Epoch 8/15
19/19 [==============================] - 0s 4ms/step - loss: 0.5499 - accuracy: 0.7459
Epoch 9/15
19/19 [==============================] - 0s 4ms/step - loss: 0.5415 - accuracy: 0.7426
Epoch 10/15
19/19 [==============================] - 0s 7ms/step - loss: 0.5357 - accuracy: 0.7393